In [2]:
import re
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError, Request
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer



# http 403 에러
def get_html(url):
    headers = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"}

    html = ""
    ret = requests.get(url, headers=headers)

    if ret.status_code == 200:
        html = ret.text

    return html



def link_text(bsObj):
    result = ""

    allText = bsObj.findAll("a")

    for link in allText:
        if 'href' in link.attrs:
            # print(link)
            result += " " + link.get_text()

    return result;



# title 텍스트
def title(bsObj):

    if bsObj.head is None:
        print("no header")
        return ""
    elif bsObj.head.title is None:
        return ""
    else:
        return bsObj.head.title.get_text()


# description 텍스트
def description(bsObj):

    result = ""

    if bsObj.head is None:
        return result

    else:
        for meta in bsObj.head.find_all('meta'):
            if meta.get('name') == "description":
                result += " " + meta.get('content')

        return result


# 키워드
def keyword(bsObj):

    result = ""
    if bsObj.head is None:
        return result

    else:
        for meta in bsObj.head.find_all('meta'):
            if meta.get('name') == "keywords":
                result += " " + str(meta.get('content'))

        return result


# 모든 메타데이터
def meta_data(bsObj):
  

    result = ""
    metadatas = set()

    if bsObj.head is None:
        return result
    else:
        for meta in bsObj.head.find_all('meta'):
            # print(meta.get('content'))
            metadatas.add(meta.get('content'))

        for data in metadatas:
            result += " " + str(data)

        return result


def page_text(url):

    html = get_html(url)
    bsObj = BeautifulSoup(html, "html.parser")

    title2 = title(bsObj)
    link_text2 = link_text(bsObj)
    keyword2 = keyword(bsObj)
    description2 = description(bsObj)
    meta_data2 = meta_data(bsObj)

    return (title2 + link_text2 + keyword2 + description2)
#     return (title2 + link_text2 + meta_data2)
    
    
print(page_text("https://www.casinogates.co.uk/"))

In [3]:
import nltk

nltk.download('stopwords')

review = page_text("http://www.newcasinos.co/")

def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()	

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()

    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)
        
#         print('num of words: ', len(clean_review.split()))

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review
  
print(preprocessing(review, True))







[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
new online casinos new casinos home new casinos new casinos mobby slots larry casino new casinos rizk casino casino casillion tom casino unique casino everum casino casino girl dunder casino cashmio viks casino new casinos casino land kaboo casino lapalingo insta casino sugar casino omni slots top casinos search software netent microgaming nextgen igt novomatic neogames boss media elk studios saucify aristocrat playtech greentube cryptologic leander games ashgaming blueprint gaming yggdrasil electracade isoftbet x gaming ezugi gaming edict bally merkur gaming real time gaming game art play n go evolution quickspin betsoft endorphina amatic wms skillonnet genesis thunderkick nextgen evolution license uk malta curacao costa rica languages english polish turkisch norwegian swedish finnish russian spanish french german currency euro us dollar uk pound south african rand tu

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import csv

DATA_PATH = '/content/gdrive/My Drive/projects/web classification/train_test2.csv'
page_data = pd.read_csv(DATA_PATH, header = None)
rows = page_data.shape[0]
# print(page_data[0])

clean_train_html = []
for page in page_data[1]:
  clean_train_html.append(page)


# 전처리한 데이터 출력
print(page_data)
print(clean_train_html)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_html)
text_sequences = tokenizer.texts_to_sequences(clean_train_html)


# print(clean_train_html)  
# print(preprocessing(page_data[0][0]))
# print(text_sequences[0])
# print(preprocessing(page_data[0][1]))
# print(text_sequences[1]) 


word_vocab = tokenizer.word_index
print(word_vocab)
print("전체 단어 개수: ", len(word_vocab) + 1)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
data_configs = {}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

In [0]:
MAX_SEQUENCE_LENGTH = 100
train_inputs = pad_sequences(text_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding='post')
print('shape of train data: ', train_inputs.shape)

shape of train data:  (2081, 100)


In [0]:

clean_train_df = pd.DataFrame({'html':clean_train_html, 'gambling': page_data[2]})
train_labels = np.array(page_data[2])

print('Shape of label tensor:', train_labels.shape)
print(train_labels)

Shape of label tensor: (2081,)
[1. 1. 1. ... 0. 0. 0.]


In [0]:
DATA_IN_PATH = '/content/gdrive/My Drive/projects/web classification/data_in/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS='data_configs.json'

import os

if not os.path.exists(DATA_IN_PATH):
  os.makedirs(DATA_IN_PATH)

np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

clean_train_df.to_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, index = False)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'),ensure_ascii=False)

